In [178]:
import bs4 
import requests as req
import re
import pandas as pd
import numpy as np
import datetime

In [251]:
generate_arabic_sentences(sentence_length=4,max_sents=100000,file_name_to_save="arabic_sentences_4_dense3",min_diacritics=3)
#generate_arabic_sentences(sentence_length=3,file_name_to_save="arabic_sentences_3_huge")
#generate_arabic_sentences(sentence_length=4,file_name_to_save="arabic_sentences_4_huge")

(2, 1)
(2, 1049)
(2, 4019)
(2, 4114)
(2, 7084)
(2, 8259)
(2, 9626)
(2, 9627)
(2, 9981)
(2, 10428)
(2, 10429)
(2, 11772)
(2, 14419)
(2, 14420)
(2, 14421)
(2, 14422)
(2, 15765)
(2, 16536)
(2, 16537)
(2, 16998)
(2, 17453)
(2, 17454)
(2, 17897)
(2, 18591)
(2, 18725)
(2, 18726)
(2, 19774)
(2, 19775)
(2, 20184)
(2, 21076)
(2, 21077)
(2, 22942)
(2, 22943)
(2, 23147)
(2, 23148)
(2, 25190)
(2, 26365)
(2, 29335)
(2, 31424)
(2, 34394)
(2, 37364)
(2, 37899)
(2, 38899)
(2, 40275)
(2, 42305)
(2, 43994)
(2, 46025)
(2, 46545)
(2, 47780)
(2, 49587)
(2, 50768)
(2, 52438)
(2, 55408)
(2, 56775)
(2, 57129)
(2, 59218)
(2, 59219)
(2, 59220)
(2, 60873)
(2, 61279)
(2, 61280)
(2, 62881)
(2, 63216)
(2, 63217)
(2, 64581)
(2, 64816)
(2, 66485)
(2, 67233)
(2, 68831)
(2, 68832)
(2, 69557)
(2, 69558)
(2, 70156)
(2, 70157)
(2, 70158)
(2, 70346)
(2, 71618)
(2, 72461)
(2, 72462)
(2, 72463)
(2, 72772)
(2, 72773)
(2, 74235)
(2, 74935)
(2, 74936)
(2, 74937)
(2, 76426)
(2, 76427)
(2, 77276)
(2, 77277)
(2, 77278)
(2, 77279)


In [248]:
def generate_arabic_sentences(sentence_length,
                              base_link="http://www.baheth.info/all.jsp?term=",
                              word_list=get_words("http://www.baheth.info/all.jsp?term=عمر"),
                              max_sents=1000000,
                              file_name_to_save="sentences"+str(datetime.datetime.now()),
                              include_index=False,
                              min_diacritics=1):
    
    #generate links
    links = generate_links(base_link,word_list)
    
    #get all sents
    all_sentences = np.array([[],[]])
    for link in links:
        conc = get_sents_concatenated(link,sentence_length,min_diacritics)
        all_sentences = append(all_sentences,conc)
        print(all_sentences.shape)
        if(all_sentences.shape[1] > max_sents):
            break
    
    #save
    all_sents = np.transpose(all_sentences)
    save_list(all_sents,file_name_to_save,include_index=include_index)
    

In [229]:
def generate_links(base,terms):
    links = []
    for term in terms:
        link = base+term
        links.append(link)
    return links

In [230]:
def get_words(link):
    text = get_arabic_text(link)
    words = clean_words(text.split(" "))
    words = remove_diacritics_from_list(words)
    return words

In [231]:
def get_sents_concatenated(link,length=4,min_diacritics=1):
    sents = get_sents(link,length,min_diacritics)
    sents_nod = remove_diacritics_from_list(sents)
    sents_all = concat(sents_nod,sents)
    return sents_all

In [232]:
def get_sents(link,length=4,min_diacritics=1):
    text = get_arabic_text(link)
    sents = split_to_sentences(text,length)
    sents_clean = clean_sents(sents,length,min_diacritics)
    return sents_clean

In [233]:
def concat(list_a,list_b):
    c = np.array([list_a,list_b])
    return c

In [234]:
def append(mat_a,mat_b):
    a_b_0 = np.append(mat_a[0],mat_b[0])
    a_b_1 = np.append(mat_a[1],mat_b[1])
    c = concat(a_b_0,a_b_1)
    return c

In [235]:
def contains_diacritics(word):
    regex = "[\u0618-\u061A|\u064B-\u0653]"
    return re.search(regex,word)

In [236]:
def num_of_diacritics(sentence):
    words = sentence.split(" ")
    regex = "[\u0618-\u061A|\u064B-\u0653]"
    num = 0
    for word in words:
        if(re.search(regex,word)):
            num += 1
    
    return num

In [237]:
def clean_sents(sents,length=4,min_diacritics=1):
    
    new_sents = []
    
    for sent in sents:
                
        #remove sents less than length 
        if(len(sent.split(" ")) != length):
            continue
        
        #remove sents that have no diacritics
        if(num_of_diacritics(sent) < min_diacritics):
            continue
        
        #add finally
        new_sents.append(sent)
    
    return new_sents

In [238]:
def split_to_sentences(text,length=4):
    
    words = clean_words(text.split(" "))
    sentences = []
    sent = words[0]
    
    for i in range(1,len(words)):
        
        if(i%length==0 or i==len(words)-1):
            sentences.append(sent.strip())
            sent = ""
        
        sent += " "+words[i]
    
    return sentences

In [239]:
def clean_words(words):
    new_words = []
    
    for word in words:
        if(len(word.strip()) > 1 and len(word.strip()) < 15 and "|" not in word):
            new_words.append(word)
            
    return new_words

In [240]:
def get_arabic_text(link):
    
    if(not check_url(link) or link == None):
        return set()
    
    #request
    res = req.get(link)
        
    #get arabic words
    ar_text = re.sub('((?![\u0621-\u0655| ]).)',"",res.text)
    ar_text = ar_text.replace("\n","")
    ar_text = ar_text.replace("\s","")
    
    return ar_text

In [241]:
def cramp_page(link):

    if(not check_url(link) or link == None):
        return set()
    
    #request
    res = req.get(link)
        
    #get arabic words
    ar_text = re.sub('((?![\u0621-\u0655| ]).)',"",res.text)
    ar_text = ar_text.replace("\n","")
    ar_text = ar_text.replace("\s","")
    ar_list = ar_text.split(" ")
    ar_set = set(ar_list)
    
    return ar_set

In [242]:
def get_nested_links(home_link,link):
                    
    if(not check_url(link)):
        return link
    
    #request
    res = req.get(link)
    
    #get page links
    soup = bs4.BeautifulSoup(res.text)
    ais = soup.find_all('a')
    hrefs = [a.get('href') for a in ais]
    hrefs = set(hrefs) # unique nested links
    
    valid_links = set()
        
    for href in hrefs:
        # add the href if it is valid 
        if(href == None or len(href) < 1):
            continue
        elif(not href.startswith('/')):
            if(not href.startswith(base_link)):
                continue
            else:
                if(check_url(href)):
                    valid_links.add(href)
        else:
            aLink = home_link + href
            if(check_url(aLink)):
                valid_links.add(aLink)
    
    return valid_links

In [243]:
def remove_diacritics_from_list(alist):
    new_list = []
    for word in alist:
        word_nod = remove_diacritis(word)
        new_list.append(word_nod)
    return new_list

In [244]:
def remove_diacritis(word):
    regex = "[\u0618-\u061A|\u064B-\u0653]" #unicode diacritis
    word_no = re.sub(regex,"",word)
    return word_no

In [245]:
def save_list(aList,name,enc="utf-8",include_index=False):
    list_pd = pd.DataFrame(aList)
    name = "./"+name+".csv"
    list_pd.to_csv(path_or_buf=name,encoding=enc,columns=[0,1],header=["no_diacritics","with_diacritics"],index=include_index)

In [246]:
def check_url(url):
    
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    
    return re.match(regex, url)